In [1]:
import requests
import pandas as pd
import itertools
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
# IEX Cloud API token
api_token = 'pk_b23ba153ee2045aeb5ca77fe63ab5cb0'

# Define the base URL for the IEX Cloud API
base_url = 'https://cloud.iexapis.com/stable'

# Define the symbol for S&P 500
symbol = 'SPY'

# Define the date as May 10, 2024
yesterday_date = '20240510'

# Define the historical endpoint
endpoint = f'/stock/{symbol}/chart/date/{yesterday_date}'

# Set parameters
params = {
    'token': api_token,
}

# Make the API request
response = requests.get(f'{base_url}{endpoint}', params=params)

# Check if the request was successful Debugging
if response.status_code == 200:
    data = response.json()
    
    # Check if data is not empty
    if data:
        # Convert the data to a DataFrame
        df = pd.DataFrame(data)
    else:
        print("No data available for yesterday.")
else:
    print(f"Error: {response.status_code}, {response.text}")

In [3]:
# Data preparation
df=df[['minute','open','close']].dropna(subset=['open', 'close'])

In [4]:
# Re-indexing
df.index = range(1, len(df) + 1)

In [5]:
# Limiting data sample
df=df.head(100)

In [6]:
df

,minute,open,close
1,09:30,521.800,521.365
2,09:31,521.450,521.730
3,09:32,521.950,521.845
4,09:33,521.860,521.875
5,09:35,521.915,522.130
...,...,...,...
96,11:15,520.445,520.460
97,11:16,520.490,520.430
98,11:17,520.330,520.340
99,11:18,520.250,520.250


In [7]:
# Define function for looping through strategies
def find_profitable_strategies_unique_sorted(df):
    profitable_strategies = {}
    # Looping buy prices
    for buy_index in range(len(df.index)):
        # Looping sell prices
        for sell_index in range(buy_index + 1, len(df.index)):
            # Calculating profit
            profit = df.iloc[sell_index]['close'] - df.iloc[buy_index]['open']
            if profit > 0:
                # Totaling profit of best strategies
                if buy_index not in profitable_strategies or profit > profitable_strategies[buy_index][1]:
                    profitable_strategies[buy_index] = (df.index[sell_index], profit)
    # Sort the profitable strategies by profit in descending order
    sorted_strategies = sorted(profitable_strategies.values(), key=lambda x: x[1], reverse=True)
    return [(df.index[buy_index], sell_index, profit) for buy_index, (sell_index, profit) in profitable_strategies.items()], len(profitable_strategies)

# Call the function to find profitable strategies and get the number of strategies
profitable_strategies_unique_sorted, num_strategies = find_profitable_strategies_unique_sorted(df)

# Sort the profitable strategies by profit in descending order
profitable_strategies_unique_sorted = sorted(profitable_strategies_unique_sorted, key=lambda x: x[2], reverse=True)

# Calculate total profit
total_profit = sum(strategy[2] for strategy in profitable_strategies_unique_sorted)

# Printing all profitable strategies with only one variation per buy index, sorted by profit
print("Number of Profitable Strategies:", num_strategies)
print("Total Profit of All Strategies:", total_profit)
for strategy in profitable_strategies_unique_sorted:
    print("Buy at:", strategy[0], "Sell at:", strategy[1], "Profit:", strategy[2])

Number of Profitable Strategies: 74
Total Profit of All Strategies: 19.89500000000021
Buy at: 2 Sell at: 23 Profit: 1.0699999999999363
Buy at: 53 Sell at: 64 Profit: 0.7300000000000182
Buy at: 1 Sell at: 23 Profit: 0.7200000000000273
Buy at: 4 Sell at: 23 Profit: 0.6599999999999682
Buy at: 5 Sell at: 23 Profit: 0.6050000000000182
Buy at: 6 Sell at: 23 Profit: 0.6000000000000227
Buy at: 59 Sell at: 64 Profit: 0.5750000000000455
Buy at: 3 Sell at: 23 Profit: 0.5699999999999363
Buy at: 55 Sell at: 64 Profit: 0.5599999999999454
Buy at: 58 Sell at: 64 Profit: 0.55499999999995
Buy at: 54 Sell at: 64 Profit: 0.5349999999999682
Buy at: 56 Sell at: 64 Profit: 0.5049999999999955
Buy at: 57 Sell at: 64 Profit: 0.5049999999999955
Buy at: 52 Sell at: 64 Profit: 0.5
Buy at: 51 Sell at: 64 Profit: 0.4800000000000182
Buy at: 77 Sell at: 87 Profit: 0.44500000000005
Buy at: 9 Sell at: 23 Profit: 0.44499999999993634
Buy at: 76 Sell at: 87 Profit: 0.4350000000000591
Buy at: 8 Sell at: 23 Profit: 0.4099999